In [1]:
import torch
import torch.nn as nn
from dataset import MyDataset
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, Resize, Normalize
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, cohen_kappa_score
import numpy as np
import os
import shutil
from tqdm.notebook import tqdm

In [2]:
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_transform = Compose([ToTensor(), 
                               # RandomAffine(degrees=(-5, 5), translate=(0.15, 0.15), scale=(0.85, 1), shear=10),
                               Resize((224, 224)), 
                               # ColorJitter(brightness=0.125, contrast=0.5, saturation=0.5, hue=0.05),
                               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ])
    test_transform = Compose([ToTensor(),
                              Resize((224, 224)),
                              Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    train_dataset = MyDataset(data_path="data/train_images", csv_file="train-1.csv", is_train=True, transform=train_transform)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=8, drop_last=True, shuffle=True, num_workers=4)
    val_dataset = MyDataset(data_path="data/train_images", csv_file="train-2.csv", is_train=False, transform=test_transform)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size=8, drop_last=True, shuffle=True, num_workers=4)
    
    model = mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT)
    model.classifier[3] = nn.Linear(1280, 5)
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    # optimizer = torch.optim.SGD(params=model.parameters(), lr=1e-10, momentum=0.9)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3, betas=(0.9, 0.999))
    
    if os.path.exists("trained_models/mobilenet_v3/last.pt"):
        checkpoint = torch.load("trained_models/mobilenet_v3/last.pt")
        model.load_state_dict(checkpoint["model"])
        optimizer.load_state_dict(checkpoint["optimizer"])
        start_epoch = checkpoint["epoch"]
        best_acc = checkpoint["best_acc"]
    else:
        start_epoch = 0
        best_acc = -1
        
    if os.path.isdir("tensorboard/mobilenet_v3"):
        shutil.rmtree("tensorboard/mobilenet_v3")
    os.makedirs("tensorboard/mobilenet_v3")
    # len(train_dataloader) = 1831, num_iters_per_epoch = 229 when batch_size = 8
    num_iters_per_epoch = len(train_dataloader)
    print(num_iters_per_epoch)
    writer = SummaryWriter("tensorboard/mobilenet_v3")
    for epoch in range(start_epoch, 400):

        # TRAINING
        model.train()
        train_loss = []
        train_labels = []
        train_predictions = []
        avg_loss= 0
        progress_bar = tqdm(train_dataloader, colour="green")
        for iter, (images, labels) in enumerate(progress_bar):
            images = images.to(device)
            labels = labels.to(device)
            predictions = model(images)
            loss = criterion(predictions, labels)
            predictions_class = torch.argmax(predictions, dim=1)
            train_labels.extend(labels.tolist())
            train_predictions.extend(predictions_class.tolist())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            avg_loss = np.mean(train_loss)
            progress_bar.set_description(" Epoch: {}/400. Loss: {:.5f}".format(epoch+1, avg_loss))
            writer.add_scalar("Train/Loss", avg_loss, epoch * num_iters_per_epoch + iter)
        acc_train = accuracy_score(train_labels, train_predictions)
        cohen_train = cohen_kappa_score(train_labels, train_predictions)
        print("Avg_loss_train: {:.5f}, Acc_train: {:.5f}, Cohen_train: {:.5f}".format(avg_loss, acc_train, cohen_train))
        
        # VALIDATION
        model.eval()
        progress_bar = tqdm(val_dataloader, colour="yellow")
        all_labels = []
        all_predictions = []
        all_losses = []
        with torch.no_grad():
            for images, labels in progress_bar:
                images = images.to(device)
                labels = labels.to(device)
                prediction = model(images)
                loss = criterion(prediction, labels)
                predicted_class = torch.argmax(prediction, dim=1)
                all_labels.extend(labels.tolist())
                all_predictions.extend(predicted_class.tolist())
                all_losses.append(loss.item())
        loss = np.mean(all_losses)
        acc = accuracy_score(all_predictions, all_labels)
        cohenscore = cohen_kappa_score(all_predictions, all_labels)
        print("Val_loss: {:.5f}, Val_acc: {:.5f}, Val_cohen_kappa_score: {:.5f}".format(loss, acc, cohenscore))
        writer.add_scalar("Val/Loss", loss, epoch)
        writer.add_scalar("Val/Acc", acc, epoch)

        # Save checkpoint
        checkpoint = {
            "epoch": epoch+1,
            "best_acc": best_acc,
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict()
        }
        torch.save(checkpoint, os.path.join("trained_models/mobilenet_v3", "last.pt"))
        if acc > best_acc:
            best_acc = acc
            torch.save(checkpoint, os.path.join("trained_models/mobilenet_v3", "best.pt"))

if __name__ == '__main__':
    train()


C:\Users\Dell\AppData\Local\Temp\ipykernel_22880\734104322.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("trained_models/mobilenet_v3/last.pt"

364


  0%|          | 0/364 [00:00<?, ?it/s]

Avg_loss_train: 0.02771, Acc_train: 0.98626, Cohen_train: 0.97928


  0%|          | 0/91 [00:00<?, ?it/s]

...

Avg_loss_train: 0.04760, Acc_train: 0.98043, Cohen_train: 0.97049


  0%|          | 0/91 [00:00<?, ?it/s]

Val_loss: 1.95362, Val_acc: 0.78709, Val_cohen_kappa_score: 0.67568


  0%|          | 0/364 [00:00<?, ?it/s]


KeyboardInterrupt

